In [ ]:
import numpy as np                                # linear algebra
import matplotlib.pyplot as plt                   # plotting
from matplotlib.animation import FuncAnimation    # animation

epsM = np.finfo(float).eps                        # machine epsilon


In [ ]:
t = np.linspace(0, 20, 11)
y = [0.00, 3.55, 3.82, 2.98, 2.32, 1.48, 1.02, 0.81, 0.41, 0.42, 0.15]
data = np.array([t, y]).T

print('  t \t  y')
print('-'*18)
print(data)
print(f'\nData shape: {data.shape}')

In [ ]:
def Residuals(x, data):
    t = data[:, 0]
    y = data[:, 1]
    return y - x[2] * np.exp(x[0] * t) - x[3] * np.exp(x[1] * t)

In [ ]:
def Func(x, data):
    f = Residuals(x, data)
    sum_square = np.dot(f.T, f)
    return 0.5 * sum_square

In [ ]:
def Fprime(x, data):
    m = data.shape[0]
    n = x.size
    t = data[:, 0]
    J = np.zeros((m, n))
    J[:, 0] = - x[2] * t * np.exp(x[0] * t)
    J[:, 1] = - x[3] * t * np.exp(x[1] * t)
    J[:, 2] = - np.exp(x[0] * t)
    J[:, 3] = - np.exp(x[1] * t)
    f = Residuals(x, data)
    return np.dot(J.T, f), J

In [ ]:
def InitMiu(J0, tau=1e-3):
    A0 = np.dot(J0.T, J0)
    mask = np.zeros(A0.shape, dtype=bool)
    np.fill_diagonal(mask, True)
    max_diag = A0[mask].max()
    return tau * max_diag, A0

In [ ]:
def HybridMethod(x, data, eps1=1e-5, eps2=1e-5, kmax=1000):
    # initial variables
    total_mu = []
    k = 0
    F = Func(x, data)
    fprime, J = Fprime(x, data)
    miu, A = InitMiu(J)
    B = np.eye(x.size)
    found = np.linalg.norm(fprime, np.inf) <= eps1
    better = False
    method = 'LM'
    
    # additional variables
    nu = 2
    count = 0
    result = []
    
    # loop until converges
    while (not found) and (k < kmax):
        k += 1
        
        # pick method
        if method == 'LM':
            xnew, Fnew, fprimenew, Jnew, found, better, methodnew, miu, delta, nu, count = \
                LMstep(x, data, F, fprime, J, A, found, better, miu, nu, count, eps1, eps2)
        if method == 'QN':
            xnew, Fnew, fprimenew, Jnew, found, better, methodnew, miu, delta = \
                QNstep(x, data, F, fprime, J, B, found, better, miu, delta, eps1, eps2)
        
        # update B
        h = xnew - x
        fnew = Residuals(xnew, data)
        y = np.dot(np.dot(Jnew.T, Jnew), h) + np.dot((Jnew - J).T, fnew)
        if np.dot(h.T, y) > 0:
            v = np.dot(B, h)
            yscaled = 1 / np.dot(h.T, y) * y
            vscaled = 1 / np.dot(h.T, v) * v
            B = B + \
                np.dot(yscaled.reshape(-1,1), y.reshape(1,-1)) - \
                np.dot(vscaled.reshape(-1,1), v.reshape(1,-1))        # reshape to maintain the dimension
        
        # update x
        if better:
            x = xnew
            F = Fnew                   
            fprime = fprimenew         
            J = Jnew                   
            A = np.dot(Jnew.T, Jnew)   
        
        fprimenorm = np.linalg.norm(fprime, np.inf)
        print('Iteration:{}\tx = {}, \n\t\tgradient = {:.4f}, step = {}'
              .format(k, x, fprimenorm, method))
        
        # update method
        method = methodnew
        result.append(x)
        total_mu.append(Fnew)
    
    # maximum iteration reached
    if k == kmax:
        print('Does not converge after {} iterations.'.format(kmax),
              'Try increasing max iteration or initializing with different point.')

    return total_mu

In [ ]:
def UpdateMiu(rho, miu, nu, method='Nielsen'):
    if method == 'Marquardt':
        if rho < 0.25:
            miu = miu * 2
        elif rho > 0.75:
            miu = miu / 3
    if method == 'Nielsen':
        if rho > 0:
            miu = miu * max(1/3, 1 - (2 * rho - 1) ** 3)
            nu = 2
        else:
            miu = miu * nu
            nu = 2 * nu
    return miu, nu

In [ ]:
def LMstep(x, data, F, fprime, J, A, found, better, miu, nu, count, eps1, eps2):
    # current variables
    xnew = x
    Fnew, fprimenew, Jnew = F, fprime, J
    method = 'LM'
    
    # solve hlm
    I = np.eye(len(A))
    hlm = - np.dot(np.linalg.inv(A + miu * I), fprime)
    hlmnorm = np.linalg.norm(hlm)
    hlmnormthres = eps2 * (np.linalg.norm(x) + eps2)
    
    # calculate delta, used when switch to QN method
    delta = max(1.5 * hlmnormthres, 0.2 * hlmnorm)
    
    # core algorithm
    if hlmnorm <= hlmnormthres:
        found = True
    else:
        # update x
        xnew = x + hlm
        Fnew = Func(xnew, data)
        fprimenew, Jnew = Fprime(xnew, data)
        fprimenewnorm = np.linalg.norm(fprimenew, np.inf)
        
        # calculate rho
        ared = F - Fnew
        pred = 1/2 * np.dot(hlm.T, miu * hlm - fprime)
        rho = ared / pred
        
        # make decision based on rho
        if rho > 0:
            better = True
            found = fprimenewnorm <= eps1
            if fprimenewnorm < 0.02 * Fnew:
                count += 1
                if count == 3:
                    method = 'QN'
            else:
                count = 0
        else:
            count = 0
            better = False
        miu, nu = UpdateMiu(rho, miu, nu)
    
    return xnew, Fnew, fprimenew, Jnew, found, better, method, miu, delta, nu, count

In [ ]:
def UpdateDelta(rho, delta, h):
    if rho < 0.25:
        delta = delta / 2
    elif rho > 0.75:
        delta = max(delta, 3 * np.linalg.norm(h))
    return delta

In [ ]:
def QNstep(x, data, F, fprime, J, B, found, better, miu, delta, eps1, eps2):
    # current variables
    xnew = x
    Fnew, fprimenew, Jnew = F, fprime, J
    method = 'QN'
    fprimenorm = np.linalg.norm(fprime, np.inf)
    better = False
    
    # solve hqn
    hqn = - np.dot(np.linalg.inv(B), fprime)
    hqnnorm = np.linalg.norm(hqn)
    hqnnormthres = eps2 * (np.linalg.norm(x) + eps2)
    
    # core algorithm
    if hqnnorm <= hqnnormthres:
        found = True
    else:
        # step length is too far
        if hqnnorm > delta:
            hqn = (delta / hqnnorm) * hqn
        
        # update x
        xnew = x + hqn
        Fnew = Func(xnew, data)
        fprimenew, Jnew = Fprime(xnew, data)
        fprimenewnorm = np.linalg.norm(fprimenew, np.inf)
        
        # calculate rho to update delta
        ared = F - Fnew
        pred = 1/2 * np.dot(hqn.T, miu * hqn - fprime)
        rho = ared / pred
        delta = UpdateDelta(rho, delta, hqn)
        
        # make decision
        if fprimenewnorm <= eps1:
            found = True
        else:
            better = (Fnew < F) or (Fnew <= (1 + np.sqrt(epsM)) * F and fprimenewnorm < fprimenorm)
            if fprimenewnorm >= fprimenorm:
                method = 'LM'
    
    return xnew, Fnew, fprimenew, Jnew, found, better, method, miu, delta

In [ ]:
x = np.array([-1,1,-10,10])
result = HybridMethod(x, data)
# print('\nFinal estimated parameters:', result[-1])

In [ ]:
result

In [ ]:
arr = np.array(result)
arr = arr[(arr >=0.2) & (arr <= 600)]
m = -np.sort(-np.random.choice(arr, 20))

In [ ]:
plt.figure(figsize=(6, 6))
# plt.plot(loss_train, 'r');
plt.semilogy(m, 'b');
plt.grid();
# plt.legend(['Train', 'Test']);
plt.xlabel('# of epoch')
plt.ylabel('Loss')
plt.title('In logarithm')

In [ ]:
# part (1)

x1 = np.arange(-1.0, 10.1, 0.05)
x2 = np.arange(-1.0, 10.1, 0.05)
# l = np.arange(-5.0, 10.1, 0.1)
l = 1
f = x1
c = abs(x1)+ abs(x2) - 1

x1_s = -1
f_s = x1_s
Lx = f - (l*c)
Lx = -np.ones(len(c))

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(f, c, 'b');
plt.plot(f, Lx, 'r');
plt.grid();
# plt.legend(['Train', 'Test']);
plt.xlabel('constraint')
plt.ylabel('cost function')
plt.title('Part (b)')
plt.legend(['[f(x), c(x)]', 'opt_lambda'])